In [ ]:
# Replace with your actual item ID
item_id = ""  #datapipeline
#item_id = "" #copyjob

In [ ]:
import requests
import json
import sempy.fabric as fabric


# Get the current workspace ID dynamically
workspace_id = fabric.get_notebook_workspace_id()

# Get the current user's access token
access_token = notebookutils.credentials.getToken('pbi')

# Construct the API URL Jobs Instances
url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items/{item_id}/jobs/instances"


# Set headers
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Make the API request
response = requests.get(url, headers=headers)


# Check and process the response
if response.status_code == 200:
    data = response.json()
    job_instances = data.get("value", []) # Extract the list of job instances
    if job_instances:
        
        json_lines = [json.dumps(job_instances) for job_instance in job_instances]
        
        df = spark.read.json(spark.sparkContext.parallelize(json_lines))
        #print('df about to run')
        #df.show(truncate=False)
        #display(df)
    else:
        print("No job instances found.")
else:
    print(f"Failed to fetch job status: {response.status_code} - {response.text}")


In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType

# Ensure startTimeUtc is in timestamp format
df = df.withColumn("startTimeUtc", col("startTimeUtc").cast(TimestampType()))

# Sort by startTimeUtc descending
df_sorted = df.orderBy(col("startTimeUtc").desc())

# Filter out rows where status is 'Deduped'
df_filtered = df_sorted.filter(col("status") != "Deduped")

# Get the first non-Deduped status
latest_status = df_filtered.select("status").first()["status"]

#print(f"The latest non-Deduped status is: {latest_status}")


In [ ]:
notebookutils.notebook.exit(latest_status)